# Análise dos dados

In [0]:
df_dim_livro = spark.read.format('delta').load('abfss://livros@recomendador.dfs.core.windows.net/gold/dim_livro')
df_dim_autor = spark.read.format('delta').load('abfss://livros@recomendador.dfs.core.windows.net/gold/dim_autor')
df_dim_genero = spark.read.format('delta').load('abfss://livros@recomendador.dfs.core.windows.net/gold/dim_genero')
df_dim_grupo = spark.read.format('delta').load('abfss://livros@recomendador.dfs.core.windows.net/gold/dim_grupo_genero')
df_bg_grupo_genero = spark.read.format('delta').load('abfss://livros@recomendador.dfs.core.windows.net/gold/bg_grupo_genero')
df_fatos_livro = spark.read.format('delta').load('abfss://livros@recomendador.dfs.core.windows.net/gold/fatos_livro')

## Criar views

In [0]:
df_dim_livro.createOrReplaceTempView('dim_livro')
df_dim_autor.createOrReplaceTempView('dim_autor')
df_dim_genero.createOrReplaceTempView('dim_genero')
df_dim_grupo.createOrReplaceTempView('dim_grupo_genero')
df_bg_grupo_genero.createOrReplaceTempView('bg_grupo_genero')
df_fatos_livro.createOrReplaceTempView('fatos_livro')

## Fazer análises

In [0]:
%sql
SELECT d.genero, COUNT(*) AS quantidade_livros
FROM fatos_livro AS f
JOIN bg_grupo_genero AS g
  ON f.id_grupo = g.id_grupo
JOIN dim_genero AS d
  ON g.id_genero = d.id
GROUP BY d.genero
ORDER BY quantidade_livros DESC
LIMIT 10

In [0]:
%sql
SELECT d.genero, MEAN(f.numero_de_estrelas) AS media_estrelas, COUNT(*) AS quantidade_livros
FROM fatos_livro AS f
JOIN bg_grupo_genero AS g
  ON f.id_grupo = g.id_grupo
JOIN dim_genero AS d
  ON g.id_genero = d.id
GROUP BY d.genero
ORDER BY media_estrelas DESC, quantidade_livros DESC
LIMIT 10

In [0]:
%sql
SELECT d.genero, MEAN(f.numero_de_estrelas) AS media_estrelas, COUNT(*) AS quantidade_livros
FROM fatos_livro AS f
JOIN bg_grupo_genero AS g
  ON f.id_grupo = g.id_grupo
JOIN dim_genero AS d
  ON g.id_genero = d.id
GROUP BY d.genero
ORDER BY quantidade_livros DESC, media_estrelas DESC
LIMIT 10

In [0]:
%sql
SELECT d.genero,
  MEAN(f.numero_de_estrelas) AS media_estrelas, SUM(f.total_de_avaliacoes) AS total_avaliacoes, COUNT(*) AS quantidade_livros
FROM fatos_livro AS f
JOIN bg_grupo_genero AS g
  ON f.id_grupo = g.id_grupo
JOIN dim_genero AS d
  ON g.id_genero = d.id
GROUP BY d.genero
ORDER BY total_avaliacoes DESC, quantidade_livros DESC, media_estrelas DESC
LIMIT 10

In [0]:
%sql
SELECT DISTINCT l.titulo, f.numero_de_estrelas AS media_estrelas, f.total_de_avaliacoes AS total_avaliacoes
FROM fatos_livro AS f
JOIN dim_livro AS l
  ON f.id_livro = l.id
ORDER BY total_avaliacoes DESC, media_estrelas DESC
LIMIT 10

In [0]:
%sql
WITH top_livros AS (
  SELECT ROW_NUMBER() OVER (
      PARTITION BY l.titulo
      ORDER BY f.total_de_avaliacoes DESC, f.numero_de_estrelas DESC
    ) AS indice_titulo,
  l.titulo,
  f.numero_de_estrelas,
  f.total_de_avaliacoes
  FROM dim_livro AS l
  JOIN fatos_livro AS f
    ON f.id_livro = l.id
  )

SELECT t.titulo, t.numero_de_estrelas
FROM top_livros AS t
WHERE t.indice_titulo = 1
ORDER BY t.total_de_avaliacoes DESC, t.numero_de_estrelas DESC
LIMIT 20

In [0]:
%sql
WITH top_generos AS (
  SELECT
    d.genero,
    MEAN(f.numero_de_estrelas) AS media_estrelas,
    SUM(f.total_de_avaliacoes) AS total_avaliacoes,
    COUNT(*) AS quantidade_livros
  FROM fatos_livro AS f
  JOIN bg_grupo_genero AS g
    ON f.id_grupo = g.id_grupo
  JOIN dim_genero AS d
    ON g.id_genero = d.id
  GROUP BY d.genero
  ORDER BY total_avaliacoes DESC, quantidade_livros DESC, media_estrelas DESC
),
  top_livros AS (
    SELECT
    ROW_NUMBER() OVER (
      PARTITION BY l.titulo
      ORDER BY t.total_avaliacoes DESC, t.media_estrelas DESC
    ) AS indice_titulo,
    l.titulo,
    f.numero_de_estrelas AS media_estrelas_livro,
    f.total_de_avaliacoes AS total_avaliacoes_livro,
    d.genero,
    t.total_avaliacoes,
    t.media_estrelas
  FROM dim_livro AS l
  JOIN fatos_livro AS f
    ON f.id_livro = l.id
  JOIN bg_grupo_genero AS g
    ON f.id_grupo = g.id_grupo
  JOIN dim_genero AS d
    ON g.id_genero = d.id
  JOIN top_generos AS t
    ON d.genero = t.genero
  )

SELECT t.titulo, t.genero
FROM top_livros AS t
WHERE t.indice_titulo = 1
ORDER BY t.total_avaliacoes_livro DESC, t.media_estrelas_livro DESC

In [0]:
%sql
WITH top_generos AS (
  SELECT
    d.genero,
    MEAN(f.numero_de_estrelas) AS media_estrelas,
    SUM(f.total_de_avaliacoes) AS total_avaliacoes,
    COUNT(*) AS quantidade_livros
  FROM fatos_livro AS f
  JOIN bg_grupo_genero AS g
    ON f.id_grupo = g.id_grupo
  JOIN dim_genero AS d
    ON g.id_genero = d.id
  GROUP BY d.genero
  ORDER BY total_avaliacoes DESC, quantidade_livros DESC, media_estrelas DESC
),
  top_livros AS (
    SELECT
    l.titulo,
    f.numero_de_estrelas AS media_estrelas_livro,
    f.total_de_avaliacoes AS total_avaliacoes_livro,
    d.genero
  FROM dim_livro AS l
  JOIN fatos_livro AS f
    ON f.id_livro = l.id
  JOIN bg_grupo_genero AS g
    ON f.id_grupo = g.id_grupo
  JOIN dim_genero AS d
    ON g.id_genero = d.id
  JOIN top_generos AS t
    ON d.genero = t.genero
  )

SELECT t.titulo, t.genero
FROM top_livros AS t
ORDER BY t.total_avaliacoes_livro DESC, t.media_estrelas_livro DESC
LIMIT 82

In [0]:
%sql
WITH top_generos AS (
  SELECT
    d.genero,
    MEAN(f.numero_de_estrelas) AS media_estrelas,
    SUM(f.total_de_avaliacoes) AS total_avaliacoes,
    COUNT(*) AS quantidade_livros,
    f.data_de_publicacao
  FROM fatos_livro AS f
  JOIN bg_grupo_genero AS g
    ON f.id_grupo = g.id_grupo
  JOIN dim_genero AS d
    ON g.id_genero = d.id
  GROUP BY d.genero, f.data_de_publicacao
  ORDER BY total_avaliacoes DESC, quantidade_livros DESC
)

SELECT t.genero, t.data_de_publicacao, t.media_estrelas
FROM top_generos AS t
WHERE t.genero IN ('Fiction', 'Fantasy', 'Audiobook', 'Thriller', 'Classics')
  AND t.data_de_publicacao BETWEEN '2010-01-01' AND '2024-12-31'